In [1]:
# %%
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd


from pigeon import annotate
from IPython.display import display
from PIL import Image

module_path = Path(os.path.abspath(os.path.join("."))).parent.parent
sys.path.append(str(module_path))
sys.path.append('./')

from fcutils.video import get_cap_from_file, get_cap_selected_frame
from fcutils.progress import track
from fcutils.path import to_yaml

from paper import Session, Stimuli, Recording


Connecting root@127.0.0.1:3306


In [2]:

# %%
# get all trials
sessions = pd.concat([pd.DataFrame(Session & 'experiment_name="Model Based V2"'), 
            pd.DataFrame(Session & 'experiment_name="Model Based V3"')]).reset_index()

stimuli = []
for i, sess in sessions.iterrows():
    stimuli.append(pd.DataFrame(Stimuli & f'uid={sess.uid}'))
    
stimuli = pd.concat(stimuli).reset_index()
print(len(stimuli))


430


In [3]:
stimuli.head()

,index,recording_uid,uid,session_name,mouse_id,stimulus_uid,overview_frame,overview_frame_off,duration,stim_type,stim_name
0,0,190328_CA535,268,190328_CA535,CA535,190328_CA535_0,35625,35985,9.0,audio,/'WAVplayer'/'525681.174583'
1,1,190328_CA535,268,190328_CA535,CA535,190328_CA535_1,41210,41570,9.0,audio,/'WAVplayer'/'525820.788153'
2,2,190328_CA535,268,190328_CA535,CA535,190328_CA535_2,55399,55759,9.0,audio,/'WAVplayer'/'526175.536567'
3,3,190328_CA535,268,190328_CA535,CA535,190328_CA535_3,75049,75409,9.0,audio,/'WAVplayer'/'526666.783029'
4,0,190328_CA511,270,190328_CA511,CA511,190328_CA511_0,54990,55350,9.0,audio,/'WAVplayer'/'537890.530942'


In [4]:

def get_trial_frame(trial):
    rec = pd.Series((
        Recording.FilePaths & f'recording_uid="{trial.recording_uid}"'
    ).fetch1())

    path = Path(rec.overview_video)
    video_base = Path(r'Z:\swc\branco\Federico\raw_behaviour\maze\video')
    video = get_cap_from_file(video_base / path.name)

    return get_cap_selected_frame(video, trial.overview_frame + 5 * 40)


# get frames
images = {}
for i, trial in track(stimuli.iterrows(), total=len(stimuli)):
    images[trial.stimulus_uid + '_' + str(trial.overview_frame)] = get_trial_frame(trial)



Output()

In [5]:

def display_image(image):
    print(image)
    img = Image.fromarray(images[image]).resize((500, 400))
    display(img)


# %%
# annotate
annotations = annotate(
    images.keys(),
    options=['open', 'closed', 'skip'],
    display_fn=display_image
)


HTML(value='0 examples annotated, 431 examples left')

Output()

Annotation done.


In [6]:

# %%
to_yaml('mb_annotations.yaml', {k:v for k,v in annotations})
# %%
